# Müzik Türü Sınıflandırma Projesi

Bu notebook, FMA (Free Music Archive) veri setini kullanarak müzik türü sınıflandırma modeli geliştirmek için veri hazırlama ve dengeleme işlemlerini içermektedir.

## Gerekli Kütüphanelerin İçe Aktarılması
Aşağıdaki hücrede, projede kullanılacak temel Python kütüphaneleri import edilmektedir:

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import RandomOverSampler, BorderlineSMOTE
from collections import Counter

%matplotlib inline
sns.set(style='whitegrid')

## Yardımcı Fonksiyonlar

### Sınıf Dağılımı Görselleştirme Fonksiyonu
Aşağıdaki fonksiyon, veri setindeki sınıf dağılımlarını görselleştirmek için kullanılacaktır. Bu görselleştirme, veri dengesizliğini anlamamıza yardımcı olur.

In [ ]:
def plot_class_distribution(y, labels, title):
    counts = pd.Series(y).value_counts().sort_index()
    valid_indices = counts.index[counts.index < len(labels)]
    counts = counts.loc[valid_indices]
    names = labels[counts.index]

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x=names, y=counts.values, hue=names, palette='viridis', legend=False)
    ax.set_title(title)
    ax.set_xlabel('Sınıf')
    ax.set_ylabel('Sayı')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## Veri Yükleme ve Ön İşleme

Bu bölümdeki fonksiyon:
- FMA metadata dosyalarını yükler
- Gerekli sütunları seçer
- Eksik verileri temizler
- Etiketleri kodlar
- Veriyi sayısal formata dönüştürür

In [ ]:
def load_data():
    tracks_path = 'fma_metadata/tracks.csv'
    features_path = 'fma_metadata/features.csv'

    if not os.path.exists(tracks_path) or not os.path.exists(features_path):
        raise FileNotFoundError(f"Gerekli veri dosyaları bulunamadı. '{tracks_path}' ve '{features_path}' dosyalarının mevcut olduğundan emin olun.")

    tracks = pd.read_csv(tracks_path, index_col=0, header=[0,1])
    
    features = pd.read_csv(features_path, index_col=0, header=[0,1])  # Çok seviyeli başlıkla oku
    features = features.loc[:, features.columns.get_level_values(0) != 'statistics']  # 'statistics' sütunlarını kaldır
    features = features.astype(np.float32)  # Sayısal olmayan sütunları kaldırdıktan sonra float'a dönüştür

    features.index = features.index.astype(str)
    tracks.index = tracks.index.astype(str)

    genre_series = tracks[('track', 'genre_top')].dropna()
    common_index = features.index.intersection(genre_series.index)

    X = features.loc[common_index]
    y_labels = genre_series.loc[common_index]

    X = X.fillna(0).replace([np.inf, -np.inf], 0).astype(np.float32)

    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y_labels)

    print('Veriler yüklendi ve önişlendi.')
    return X, y, label_encoder

## Başlangıç Veri Analizi

Verinin ilk yüklemesini yapıp, başlangıçtaki sınıf dağılımını inceleyelim. Bu analiz, veri dengesizliği problemini görselleştirmemize yardımcı olacak.

In [ ]:
# Veriyi yükle ve önişle
X, y, le = load_data()

# Başlangıç dağılımını göster
plot_class_distribution(y, le.classes_, 'Başlangıç Sınıf Dağılımı')

## Veri Bölme ve Eğitim Seti Analizi

Veriyi eğitim ve test setlerine ayırıp, eğitim setindeki sınıf dağılımını inceliyoruz. Stratified split kullanarak orijinal dağılımı koruyoruz.

In [ ]:
# Veriyi böl ve eğitim dağılımını göster
# İlk bölme: Ana eğitim ve test setleri
X_train_orig, X_test, y_train_orig, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# İkinci bölme: Ana eğitim setini resampling ve temiz doğrulama setlerine ayır
X_train_for_resample, X_val_clean, y_train_for_resample, y_val_clean = train_test_split(
    X_train_orig, y_train_orig, test_size=0.15, stratify=y_train_orig, random_state=42
)

print(f'İlk bölünme tamamlandı: X_train_orig {X_train_orig.shape}, X_test {X_test.shape}')
print(f'İkinci bölünme tamamlandı: X_train_for_resample {X_train_for_resample.shape}, X_val_clean {X_val_clean.shape}')

plot_class_distribution(y_train_for_resample, le.classes_, 'Resampling İçin Eğitim Seti Dağılımı')

# Detaylı dağılımı yazdır
unique, counts = np.unique(y_train_for_resample, return_counts=True)
print("\nResampling İçin Eğitim Seti Dağılımı (ham sayılar):")
for i, (u, c) in enumerate(zip(unique, counts)):
    print(f"Sınıf {u} ({le.classes_[i]}): {c} örnek")

## Veri Dengeleme - Aşama 1

İlk aşamada, çok az örneğe sahip sınıflar için RandomOverSampler kullanılıyor. Bu aşama, BorderlineSMOTE için yeterli örnek sayısına ulaşmamızı sağlar.

In [ ]:
# Adım 1: En az temsil edilen sınıflar için RandomOverSampler
print('\nAdım 1: Aşırı az temsil edilen sınıflar için RandomOverSampler uygulanıyor...')
min_samples_threshold = 50  # BorderlineSMOTE için gereken minimum örnek sayısı
ros = RandomOverSampler(sampling_strategy={3: min_samples_threshold}, random_state=42)
X_partial, y_partial = ros.fit_resample(X_train_for_resample, y_train_for_resample)

# Ara sonuçları göster
unique_partial, counts_partial = np.unique(y_partial, return_counts=True)
print("\nRandomOverSampler sonrası dağılım (ham sayılar):")
for i, (u, c) in enumerate(zip(unique_partial, counts_partial)):
    print(f"Sınıf {u} ({le.classes_[i]}): {c} örnek")

plot_class_distribution(y_partial, le.classes_, 'RandomOverSampler Sonrası')

## Veri Dengeleme - Aşama 2

İkinci aşamada, daha sofistike bir yaklaşım olan BorderlineSMOTE kullanılarak kalan sınıflar dengeleniyor. Bu yöntem, sadece rastgele kopyalama yerine sentetik örnekler oluşturur.

Not: Bu aşama, veri setinin yapısına bağlı olarak başarısız olabilir. Bu durumda, ilk aşamadaki sonuçlar kullanılacaktır.

In [ ]:
# Adım 2: Kalan sınıflar için BorderlineSMOTE
print('\nAdım 2: Kalan sınıflar için BorderlineSMOTE uygulanıyor...')
borderline_smote = BorderlineSMOTE(random_state=42)

try:
    X_res, y_res = borderline_smote.fit_resample(X_partial, y_partial)
    print(f'Kombine örnekleme tamamlandı: X_res {X_res.shape}, y_res {y_res.shape}')
    
    # Son dağılımı yazdır ve göster
    unique_res, counts_res = np.unique(y_res, return_counts=True)
    print("\nSon Dağılım (ham sayılar):")
    for i, (u, c) in enumerate(zip(unique_res, counts_res)):
        print(f"Sınıf {u} ({le.classes_[i]}): {c} örnek")
    
    plot_class_distribution(y_res, le.classes_, 'Son Dengelenmiş Dağılım')

except Exception as e:
    print(f'BorderlineSMOTE örnekleme başarısız oldu: {e} - kısmi örneklenmiş veri kullanılıyor')
    X_res, y_res = X_partial, y_partial
    plot_class_distribution(y_res, le.classes_, 'Kısmi Örnekleme (BorderlineSMOTE başarısız)')

print("\nİşlem hattı tamamlandı. Yeniden örneklenmiş eğitim verisi (X_res, y_res) ve test verisi (X_test, y_test) hazır.")

## Özellik Seçimi (K-Best Feature Selection)

Model performansını artırmak ve aşırı öğrenmeyi (overfitting) azaltmak için K-Best özellik seçimi algoritmasını uygulayacağız. Bu algoritma, her özelliğin hedef değişkenle olan istatistiksel ilişkisini ölçer ve en anlamlı K özelliği seçer.

In [ ]:
# K-Best özellik seçimi uygulaması
print('\nK-Best özellik seçimi uygulanıyor...')

k = 250  # Seçilecek özellik sayısı
print(f"Toplam özellik sayısı: {X_res.shape[1]}, Seçilecek özellik sayısı: {k}")

# SelectKBest ile özellik seçimi - Sadece resampled data üzerinde fit et
selector = SelectKBest(score_func=f_classif, k=k)
X_res_selected = selector.fit_transform(X_res, y_res)

# Fitted selector ile validation ve test setlerini transform et
X_val_clean_selected = selector.transform(X_val_clean)
X_test_selected = selector.transform(X_test)

# Hangi özelliklerin seçildiğini gösteren görselleştirme
selected_mask = selector.get_support()
scores = selector.scores_
feature_indices = np.arange(len(selected_mask))

plt.figure(figsize=(12, 6))
plt.bar(feature_indices, scores, alpha=0.3, color='g')
plt.bar(feature_indices[selected_mask], scores[selected_mask], color='g')
plt.title('Özellik Skorları ve Seçilen Özellikler')
plt.xlabel('Özellik İndeksi')
plt.ylabel('F-değeri (F-value)')
plt.tight_layout()
plt.show()

print(f"Özellik seçimi tamamlandı. Seçilen özelliklerin boyutu: {X_res_selected.shape}")
print(f"Validation set boyutu: {X_val_clean_selected.shape}")
print(f"Test set boyutu: {X_test_selected.shape}")

# Orijinal veriyi güncellenmiş veri ile değiştirelim
X_res = X_res_selected
X_val_clean = X_val_clean_selected
X_test = X_test_selected

# Veri Ölçeklendirme (StandardScaler) - Sadece resampled data üzerinde fit et
print('\nVeri ölçeklendirme uygulanıyor...')
scaler = StandardScaler()
X_res_scaled = scaler.fit_transform(X_res)
X_val_clean_scaled = scaler.transform(X_val_clean)
X_test_scaled = scaler.transform(X_test)

print("Veri ölçeklendirme tamamlandı.")
print(f"Ölçeklenmiş resampled eğitim verisi boyutu: {X_res_scaled.shape}")
print(f"Ölçeklenmiş validation verisi boyutu: {X_val_clean_scaled.shape}")
print(f"Ölçeklenmiş test verisi boyutu: {X_test_scaled.shape}")

# Veriyi güncellenmiş ölçeklenmiş veriler ile değiştir
X_res = X_res_scaled
X_val_clean = X_val_clean_scaled
X_test = X_test_scaled

*-----------------------------------------------------------------------------------*
# PyTorch LSTM MODEL EĞİTİMİ
*-----------------------------------------------------------------------------------*

In [ ]:
print("\nPyTorch LSTM Model Eğitimi Başlıyor...")

# Veri yükleme, önişleme, bölme ve dengeleme adımlarının tamamlandığı varsayılır.
# Bu noktada aşağıdaki değişkenlerin mevcut olması beklenir:
# X_res, y_res (Dengelenmiş eğitim verisi)
# X_val, y_val (Doğrulama verisi)
# X_test, y_test (Test verisi)
# le (LabelEncoder nesnesi)

## Gelişmiş Özellik Mühendisliği ve Model Optimizasyonu (İsteğe Bağlı)

Bu bölümde, model performansını artırmak için gelişmiş özellik mühendisliği tekniklerini ve model optimizasyonlarını güvenli bir şekilde uygulayabiliriz. Bu teknikler veri sızıntısını önlemek için dikkatli bir şekilde tasarlanmıştır.

In [ ]:
# İsteğe bağlı: Güvenli Gelişmiş Özellik Mühendisliği
# Bu kısmı yalnızca mevcut pipeline'ın performansını artırmak istiyorsanız çalıştırın

def enhanced_feature_engineering_pipeline(X_train, X_val, X_test, apply_pca=True, apply_poly=False, pca_variance=0.95, max_poly_features=10):
    """
    Güvenli gelişmiş özellik mühendisliği pipeline'ı
    - Veri sızıntısını önler
    - Sadece eğitim verisi üzerinde fit edilir
    - Doğrulama ve test setleri transform edilir
    """
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import PolynomialFeatures
    
    # Orijinal veriyi backup'la
    X_train_backup = X_train.copy()
    X_val_backup = X_val.copy()
    X_test_backup = X_test.copy()
    
    enhanced_features_train = [X_train]
    enhanced_features_val = [X_val]
    enhanced_features_test = [X_test]
    
    try:
        # 1. PCA for dimensionality reduction
        if apply_pca:
            print(f"PCA uygulanıyor (variance: {pca_variance})...")
            pca = PCA(n_components=pca_variance)
            X_train_pca = pca.fit_transform(X_train)
            X_val_pca = pca.transform(X_val)
            X_test_pca = pca.transform(X_test)
            
            # Limit PCA features to prevent overfitting
            max_pca_features = min(50, X_train_pca.shape[1])
            enhanced_features_train.append(X_train_pca[:, :max_pca_features])
            enhanced_features_val.append(X_val_pca[:, :max_pca_features])
            enhanced_features_test.append(X_test_pca[:, :max_pca_features])
            
            print(f"PCA tamamlandı. {X_train_pca.shape[1]} bileşenden {max_pca_features} tanesi kullanıldı.")
        
        # 2. Polynomial features (very conservative)
        if apply_poly and max_poly_features > 0:
            print(f"Polinom özellikleri uygulanıyor (max {max_poly_features} özellik)...")
            n_features_for_poly = min(max_poly_features, X_train.shape[1])
            poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
            
            X_train_poly = poly.fit_transform(X_train[:, :n_features_for_poly])
            X_val_poly = poly.transform(X_val[:, :n_features_for_poly])
            X_test_poly = poly.transform(X_test[:, :n_features_for_poly])
            
            # Limit polynomial features
            max_poly_out = min(20, X_train_poly.shape[1])
            enhanced_features_train.append(X_train_poly[:, :max_poly_out])
            enhanced_features_val.append(X_val_poly[:, :max_poly_out])
            enhanced_features_test.append(X_test_poly[:, :max_poly_out])
            
            print(f"Polinom özellikleri tamamlandı. {X_train_poly.shape[1]} özellikten {max_poly_out} tanesi kullanıldı.")
        
        # 3. Combine all features
        X_train_enhanced = np.hstack(enhanced_features_train)
        X_val_enhanced = np.hstack(enhanced_features_val)
        X_test_enhanced = np.hstack(enhanced_features_test)
        
        # 4. Apply scaling to enhanced features
        scaler_enhanced = StandardScaler()
        X_train_final = scaler_enhanced.fit_transform(X_train_enhanced)
        X_val_final = scaler_enhanced.transform(X_val_enhanced)
        X_test_final = scaler_enhanced.transform(X_test_enhanced)
        
        print(f"Gelişmiş özellik mühendisliği başarılı!")
        print(f"Özellik boyutları: {X_train.shape[1]} -> {X_train_final.shape[1]}")
        
        return X_train_final, X_val_final, X_test_final, True
        
    except Exception as e:
        print(f"Gelişmiş özellik mühendisliği başarısız: {e}")
        print("Orijinal veriler kullanılıyor.")
        return X_train_backup, X_val_backup, X_test_backup, False

# Gelişmiş özellik mühendisliği ayarları
APPLY_ENHANCED_FEATURES = False  # Bu değeri True yaparak etkinleştirin
APPLY_PCA = True
APPLY_POLYNOMIAL = False  # Dikkatli olun, bu çok fazla özellik yaratabilir
PCA_VARIANCE = 0.95
MAX_POLY_FEATURES = 5  # Çok küçük tutun

if APPLY_ENHANCED_FEATURES:
    print("Gelişmiş özellik mühendisliği uygulanıyor...")
    
    # Mevcut verileri enhanced features ile değiştir
    X_res_enhanced, X_val_clean_enhanced, X_test_enhanced, success = enhanced_feature_engineering_pipeline(
        X_res, X_val_clean, X_test, 
        apply_pca=APPLY_PCA, 
        apply_poly=APPLY_POLYNOMIAL,
        pca_variance=PCA_VARIANCE,
        max_poly_features=MAX_POLY_FEATURES
    )
    
    if success:
        X_res = X_res_enhanced
        X_val_clean = X_val_clean_enhanced
        X_test = X_test_enhanced
        print("Gelişmiş özellikler başarıyla uygulandı.")
    else:
        print("Orijinal özellikler korundu.")
else:
    print("Gelişmiş özellik mühendisliği devre dışı. Mevcut pipeline kullanılıyor.")

print(f"\nFinal veri boyutları:")
print(f"Training (X_res): {X_res.shape}")
print(f"Validation (X_val_clean): {X_val_clean.shape}")
print(f"Test (X_test): {X_test.shape}")

# Model alternatiflerini test etme fonksiyonu
def compare_model_architectures():
    """
    Farklı model mimarilerini karşılaştırma (isteğe bağlı)
    """
    print("\n=== MODEL MİMARİSİ KARŞILAŞTIRMASI ===")
    print("Mevcut LSTM modeli yanında şu alternatifleri deneyebilirsiniz:")
    print("1. Bidirectional LSTM")
    print("2. GRU (Gated Recurrent Unit)")
    print("3. 1D CNN + LSTM hibrit")
    print("4. Transformer tabanlı model")
    print("5. Ensemble modeller")
    
    print("\nMevcut hiperparametrelerle deneme önerileri:")
    print("- Hidden size: [64, 128, 256]")
    print("- Num layers: [1, 2, 3]")
    print("- Dropout: [0.2, 0.3, 0.5]")
    print("- Learning rate: [0.001, 0.01, 0.0001]")

compare_model_architectures()

## LSTM Modeli için Veri Hazırlığı

PyTorch LSTM modeli için, veriyi uygun formata dönüştürmemiz gerekir. LSTM modeller sıralı veri bekler, bu nedenle öznitelik vektörünü zamansal bir diziye dönüştüreceğiz.

In [ ]:
# PyTorch tensörlerine dönüştürme ve veri setlerini hazırlama
def create_sequence_data(X, y, sequence_length=10):
    """
    Öznitelik vektörünü sıralı verilere dönüştürür.
    FMA veri seti sıralı yapıda değil, bu nedenle yapay bir sıra oluşturuyoruz.
    """
    # Veri boyutlarını kontrol et
    n_samples, n_features = X.shape
    
    # Veriyi yeniden şekillendirme
    features_per_timestep = n_features // sequence_length
    
    if features_per_timestep == 0:
        features_per_timestep = 1
        sequence_length = min(sequence_length, n_features)
    
    # Son timestep'e sığmayan özellikleri ele alma
    remainder = n_features - (sequence_length * features_per_timestep)
    
    # Yeniden şekillendirilmiş veri için array oluşturma
    X_seq = np.zeros((n_samples, sequence_length, features_per_timestep))
    
    # Veriyi yeniden şekillendirme
    for i in range(n_samples):
        for t in range(sequence_length):
            start_idx = t * features_per_timestep
            end_idx = min(start_idx + features_per_timestep, n_features)
            
            if start_idx < n_features:
                X_seq[i, t, :end_idx-start_idx] = X[i, start_idx:end_idx]
    
    # PyTorch tensörlerine dönüştürme
    X_tensor = torch.FloatTensor(X_seq)
    y_tensor = torch.LongTensor(y)
    
    return X_tensor, y_tensor

# Sıralı veri için hiperparametre
sequence_length = 5

# Ölçeklenmiş verileri sıralı forma dönüştürme - Yeni pipeline değişkenlerini kullan
X_train_seq, y_train_tensor = create_sequence_data(X_res, y_res, sequence_length)
X_val_seq, y_val_tensor = create_sequence_data(X_val_clean, y_val_clean, sequence_length)
X_test_seq, y_test_tensor = create_sequence_data(X_test, y_test, sequence_length)

print(f"Eğitim veri boyutu: {X_train_seq.shape}")
print(f"Doğrulama veri boyutu: {X_val_seq.shape}")
print(f"Test veri boyutu: {X_test_seq.shape}")

# PyTorch DataLoader oluşturma
batch_size = 512
train_dataset = TensorDataset(X_train_seq, y_train_tensor)
val_dataset = TensorDataset(X_val_seq, y_val_tensor)
test_dataset = TensorDataset(X_test_seq, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## LSTM Model Tanımı ve Eğitimi

Aşağıda müzik türü sınıflandırması için bir LSTM (Long Short-Term Memory) ağı tanımlıyoruz. LSTM'ler, müzik gibi sıralı verilerde başarılı olan bir derin öğrenme mimarisidir.

In [ ]:
# LSTM model sınıfını tanımlama
class MusicGenreLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.3):
        super(MusicGenreLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM katmanları
        self.lstm = nn.LSTM(
            input_size=input_size, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Batch normalization
        self.batch_norm = nn.BatchNorm1d(hidden_size)
        
        # Dropout katmanı
        self.dropout = nn.Dropout(dropout)
        
        # Tam bağlantılı katmanlar
        self.fc1 = nn.Linear(hidden_size, 128)  # İlk tam bağlantılı katman
        self.fc2 = nn.Linear(128, num_classes)
        
        # Aktivasyon fonksiyonları
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # LSTM katmanından geçirme
        # x şekli: (batch_size, sequence_length, input_size)
        lstm_out, _ = self.lstm(x)
        
        # Son zaman adımının çıktısını al
        # lstm_out şekli: (batch_size, sequence_length, hidden_size)
        lstm_out = lstm_out[:, -1, :]
        
        # Batch normalization
        batch_norm_out = self.batch_norm(lstm_out)
        
        # İlk tam bağlantılı katman
        fc1_out = self.fc1(batch_norm_out)
        fc1_out = self.relu(fc1_out)
        fc1_out = self.dropout(fc1_out)
        
        # İkinci tam bağlantılı katman (çıkış katmanı)
        out = self.fc2(fc1_out)
        
        return out

# Model parametreleri
input_size = X_train_seq.shape[2]  # Bir zaman adımındaki özellik sayısı
hidden_size = 128  # LSTM gizli katman boyutu
num_layers = 2  # LSTM katman sayısı
num_classes = len(le.classes_)  # Sınıf sayısı
dropout = 0.3

# GPU kullanılabilir mi kontrol et
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Kullanılan cihaz: {device}")

# Model oluşturma
model = MusicGenreLSTM(input_size, hidden_size, num_layers, num_classes, dropout).to(device)
print(model)

# Kayıp fonksiyonu ve optimize edici tanımlama
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# Eğitim fonksiyonu
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=50, early_stopping_patience=5, min_improvement_threshold=0.001):
    # Ölçüm değerlerini saklayacak listeler
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    
    # En iyi doğrulama kaybını ve modeli saklama
    # min_improvement_threshold: Doğrulama kaybındaki minimum iyileşme eşiği, 
    # bunun altındaki iyileşmeler anlamlı kabul edilmez ve erken durdurma sayacı sıfırlanmaz.
    best_val_loss = float('inf')
    best_model = None
    
    # Erken durdurma için sayaç ve sabır parametresi
    early_stopping_counter = 0
    
    for epoch in range(num_epochs):
        # Eğitim modu
        model.train()
        
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Gradyanları sıfırla
            optimizer.zero_grad()
            
            # İleri geçiş
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Geri yayılım ve optimize etme
            loss.backward()
            optimizer.step()
            
            # İstatistikleri güncelle
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        # Doğrulama modu
        model.eval()
        
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                # İleri geçiş
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                # İstatistikleri güncelle
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        # Epoch sonuçlarını hesapla
        epoch_train_loss = train_loss / len(train_loader.dataset)
        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_train_acc = train_correct / train_total
        epoch_val_acc = val_correct / val_total
        
        # Öğrenme oranını ayarla
        scheduler.step(epoch_val_loss)
        
        # Sonuçları sakla
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        train_accs.append(epoch_train_acc)
        val_accs.append(epoch_val_acc)
        
        # Eğitim durumunu yazdır
        print(f'Epoch {epoch+1}/{num_epochs} - '
              f'Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}, '
              f'Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}')
        
        # En iyi modeli sakla ve erken durdurma durumunu kontrol et
        # Doğrulama kaybındaki iyileşme miktarını hesapla
        improvement = best_val_loss - epoch_val_loss
        
        if epoch_val_loss < best_val_loss:
            # Eğer iyileşme miktarı eşik değerinden fazlaysa sayacı sıfırla
            if improvement > min_improvement_threshold:
                early_stopping_counter = 0  # Counter sıfırla
                print(f'Validation loss improved by {improvement:.6f}, which is above threshold ({min_improvement_threshold:.6f})')
            else:
                # İyileşme var ama eşik değerinin altında, bu durumda counter'ı artırıyoruz
                early_stopping_counter += 1
                print(f'Validation loss improved by only {improvement:.6f}, which is below threshold ({min_improvement_threshold:.6f})')
            
            # En iyi modeli ve validation loss değerini her durumda güncelle
            best_val_loss = epoch_val_loss
            best_model = model.state_dict()
        else:
            early_stopping_counter += 1  # Counter artır
            
        # Erken durdurma kontrolü
        if early_stopping_counter >= early_stopping_patience:
            print(f'Erken durdurma: Validation loss {early_stopping_patience} epoch boyunca yeterince iyileşmedi (minimum eşik: {min_improvement_threshold:.6f}).')
            break
    
    # En iyi model ağırlıklarını yükle
    model.load_state_dict(best_model)
    
    return model, train_losses, val_losses, train_accs, val_accs

# Modeli eğit
print("Model eğitimi başlıyor...")
num_epochs = 50
early_stopping_patience = 3  # Model belirli bir eşik değerinden fazla iyileşmezse, bu sayıda epoch sonra eğitimi durdur

try:
    # Doğrulama kaybında 0.02 altındaki iyileşmeleri önemsiz olarak kabul et
    min_improvement_threshold = 0.02  
    
    model, train_losses, val_losses, train_accs, val_accs = train_model(
        model, train_loader, val_loader, criterion, optimizer, scheduler, 
        num_epochs=num_epochs, early_stopping_patience=early_stopping_patience,
        min_improvement_threshold=min_improvement_threshold
    )
    print("Model eğitimi tamamlandı!")
except KeyboardInterrupt:
    print("Eğitim kullanıcı tarafından durduruldu.")

## Model Değerlendirmesi ve Görselleştirme

Bu bölümde eğitilmiş modeli test veri seti üzerinde değerlendirip, sonuçları görselleştireceğiz.

In [ ]:
# Eğitim sonuçlarını görselleştirme
def plot_training_history(train_losses, val_losses, train_accs, val_accs):
    plt.figure(figsize=(14, 5))
    
    # Kayıp grafiği
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Eğitim', marker='o')
    plt.plot(val_losses, label='Doğrulama', marker='*')
    plt.title('Model Kaybı')
    plt.xlabel('Epoch')
    plt.ylabel('Kayıp (Cross-Entropy)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    # Doğruluk grafiği
    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Eğitim', marker='o')
    plt.plot(val_accs, label='Doğrulama', marker='*')
    plt.title('Model Doğruluğu')
    plt.xlabel('Epoch')
    plt.ylabel('Doğruluk')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    plt.tight_layout()
    plt.show()

# Eğitim sonuçlarını görselleştir
try:
    plot_training_history(train_losses, val_losses, train_accs, val_accs)
except NameError:
    print("Eğitim geçmişi bulunamadı. Önce modeli eğitin.")

# Test veri seti üzerinde değerlendirme
def evaluate_model(model, test_loader, device):
    model.eval()
    
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    
    # Doğruluk hesapla
    accuracy = np.mean(np.array(y_true) == np.array(y_pred))
    
    # Sonuçları yazdır
    print(f"Test Doğruluğu: {accuracy:.4f}")
    
    # Sınıflandırma raporu
    print("\nSınıflandırma Raporu:")
    print(classification_report(y_true, y_pred, target_names=le.classes_))
    
    # Karmaşıklık matrisi
    plt.figure(figsize=(12, 10))
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title('Karmaşıklık Matrisi')
    plt.xlabel('Tahmin Edilen Etiketler')
    plt.ylabel('Gerçek Etiketler')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    return y_true, y_pred

# Test veri seti üzerinde değerlendir
try:
    y_true, y_pred = evaluate_model(model, test_loader, device)
except NameError:
    print("Model bulunamadı. Önce modeli eğitin.")

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, accuracy_score
from sklearn.preprocessing import label_binarize
import torch.nn.functional as F

def get_model_probabilities(model, test_loader, device):
    """
    Get prediction probabilities from the trained model
    """
    model.eval()
    y_true = []
    y_proba = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)
            
            y_true.extend(labels.cpu().numpy())
            y_proba.extend(probabilities.cpu().numpy())
    
    return np.array(y_true), np.array(y_proba)

def comprehensive_evaluation(model, test_loader, device, class_names):
    """
    Provide detailed evaluation metrics for the trained model
    """
    # Get true labels and prediction probabilities
    y_true, y_proba = get_model_probabilities(model, test_loader, device)
    
    # Get predictions
    y_pred = np.argmax(y_proba, axis=1)
    
    # Basic metrics
    accuracy = accuracy_score(y_true, y_pred)
    
    # Per-class metrics
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average=None, labels=range(len(class_names))
    )
    
    # Macro and weighted averages
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro'
    )
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        y_true, y_pred, average='weighted'
    )
    
    # AUC-ROC for multiclass
    y_true_binarized = label_binarize(y_true, classes=range(len(class_names)))
    auc_scores = []
    for i in range(len(class_names)):
        if len(np.unique(y_true_binarized[:, i])) > 1:  # Check if class exists
            auc = roc_auc_score(y_true_binarized[:, i], y_proba[:, i])
            auc_scores.append(auc)
    
    # Create detailed report
    results = {
        'accuracy': accuracy,
        'macro_precision': precision_macro,
        'macro_recall': recall_macro,
        'macro_f1': f1_macro,
        'weighted_precision': precision_weighted,
        'weighted_recall': recall_weighted,
        'weighted_f1': f1_weighted,
        'mean_auc': np.mean(auc_scores) if auc_scores else 0,
        'per_class_metrics': {
            class_names[i]: {
                'precision': precision[i],
                'recall': recall[i],
                'f1': f1[i],
                'support': support[i]
            } for i in range(len(class_names))
        }
    }
    
    return results

# Use the comprehensive evaluation function
try:
    if 'model' in locals() and 'test_loader' in locals():
        print("\nDetaylı model değerlendirmesi...")
        detailed_results = comprehensive_evaluation(model, test_loader, device, le.classes_)
        
        print(f"\nDetaylı Sonuçlar:")
        print(f"Accuracy: {detailed_results['accuracy']:.4f}")
        print(f"Macro F1: {detailed_results['macro_f1']:.4f}")
        print(f"Weighted F1: {detailed_results['weighted_f1']:.4f}")
        print(f"Mean AUC: {detailed_results['mean_auc']:.4f}")
        
        print("\nSınıf bazında detaylar:")
        for class_name, metrics in detailed_results['per_class_metrics'].items():
            print(f"{class_name}: F1={metrics['f1']:.3f}, Precision={metrics['precision']:.3f}, Recall={metrics['recall']:.3f}")
    else:
        print("Model henüz eğitilmemiş. Önce modeli eğitin.")
except NameError:
    print("Model bulunamadı. Önce modeli eğitin.")

## Model Değerlendirmesi ve İleriye Dönük Çalışmalar

Müzik türü sınıflandırma modelimiz veriyi dengeledikten sonra eğitilmiştir. Sonuçlar değerlendirilirken şunlar göz önünde bulundurulmalıdır:

1. **Veri Kalitesi**: FMA veri setindeki özellikler, ses dosyalarından çıkarılmış özelliklerdir. Daha iyi sonuçlar için ham ses verileri üzerinde spektrogram analizi yapılabilir.

2. **Model Mimarisi**: LSTM modeli, sıralı verilerde başarılı olmasına rağmen, müzik türü tanıma için CNN (Convolutional Neural Network) veya CNN-LSTM hibrit modeller de kullanılabilir.

3. **Hiperparametreler**: Farklı hiperparametreler (örn. öğrenme oranı, katman sayısı, nöron sayısı) ile model performansı artırılabilir.

4. **Veri Dengeleme**: Kullandığımız veri dengeleme yöntemleri, eğitim setindeki sınıf dağılımını eşitlemeye yardımcı olur, ancak sentetik veri oluşturma riskleri de taşır.

5. **Özellik Seçimi**: K-Best algoritması ile seçilen özellikler, modelin daha iyi genelleme yapmasına ve aşırı öğrenmesinin azalmasına yardımcı olabilir. Farklı K değerleri denenerek optimum özellik sayısı bulunabilir.

İleriye dönük çalışmalarda, daha karmaşık modeller, farklı özellik çıkarma teknikleri ve daha büyük veri setleri kullanılarak performans artırılabilir.

## Model Optimizasyonu ve Sorun Giderme

Bu bölüm, model performansını artırmak için çeşitli optimizasyon teknikleri ve sorun giderme yöntemlerini içerir.

In [ ]:
# Model Performans Analizi ve İyileştirme Önerileri

def analyze_model_performance():
    """
    Model performansını analiz et ve iyileştirme önerileri sun
    """
    try:
        if 'detailed_results' in locals() or 'detailed_results' in globals():
            results = detailed_results
            
            print("\n=== MODEL PERFORMANS ANALİZİ ===")
            print(f"Genel Doğruluk: {results['accuracy']:.4f}")
            print(f"Macro F1 Skoru: {results['macro_f1']:.4f}")
            print(f"Weighted F1 Skoru: {results['weighted_f1']:.4f}")
            print(f"Ortalama AUC: {results['mean_auc']:.4f}")
            
            # Performans değerlendirmesi ve öneriler
            if results['accuracy'] < 0.6:
                print("\n⚠️  DÜŞÜK PERFORMANS TESPİT EDİLDİ")
                print("Öneriler:")
                print("1. Daha fazla veri toplama")
                print("2. Farklı model mimarisi deneme (CNN, Transformer)")
                print("3. Hiperparametre optimizasyonu")
                print("4. Veri ön işleme tekniklerini gözden geçirme")
                
            elif results['accuracy'] < 0.75:
                print("\n📊 ORTA SEVİYE PERFORMANS")
                print("İyileştirme önerileri:")
                print("1. Özellik mühendisliği uygulama")
                print("2. Model ensemble teknikleri")
                print("3. Daha sofistike veri dengeleme")
                print("4. Regularization teknikleri")
                
            else:
                print("\n✅ İYİ PERFORMANS")
                print("Model başarılı bir şekilde çalışıyor.")
                
            # Sınıf bazında performans analizi
            print("\n=== SINIF BAZINDA PERFORMANS ===")
            poor_classes = []
            for class_name, metrics in results['per_class_metrics'].items():
                if metrics['f1'] < 0.5:
                    poor_classes.append(class_name)
                    
            if poor_classes:
                print(f"Düşük performanslı sınıflar: {', '.join(poor_classes)}")
                print("Bu sınıflar için:")
                print("- Daha fazla veri toplama")
                print("- Sınıf ağırlıklandırma")
                print("- Focal loss kullanma")
                
        else:
            print("Model değerlendirmesi henüz yapılmamış.")
            
    except Exception as e:
        print(f"Performans analizi sırasında hata: {e}")

def get_improvement_suggestions():
    """
    Gelişmiş iyileştirme önerileri
    """
    suggestions = {
        "Veri İyileştirmeleri": [
            "Veri temizleme ve outlier detection",
            "Feature scaling yöntemlerini karşılaştırma (RobustScaler, MinMaxScaler)",
            "Veri artırma teknikleri (audio augmentation)"
        ],
        "Model İyileştirmeleri": [
            "Bidirectional LSTM kullanma",
            "Attention mechanism ekleme",
            "Residual connections",
            "Batch normalization optimizasyonu"
        ],
        "Eğitim İyileştirmeleri": [
            "Learning rate scheduling",
            "Gradient clipping",
            "Warm-up strategies",
            "Cyclical learning rates"
        ],
        "Ensemble Yöntemleri": [
            "Farklı model mimarilerini birleştirme",
            "Voting classifiers",
            "Stacking",
            "Bagging"
        ]
    }
    
    print("\n=== GELİŞMİŞ İYİLEŞTİRME ÖNERİLERİ ===")
    for category, items in suggestions.items():
        print(f"\n{category}:")
        for item in items:
            print(f"  • {item}")

# Performans analizini çalıştır
analyze_model_performance()
get_improvement_suggestions()

print("\n" + "="*80)
print("MODEL EĞİTİMİ VE DEĞERLENDİRMESİ TAMAMLANDI")
print("="*80)

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
import copy

def pytorch_model_evaluation(model, val_loader, device):
    """
    Evaluate PyTorch model on validation set and return accuracy.
    """
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return correct / total

def cross_validate_lstm(X, y, model_params, cv_folds=3, num_epochs=10):
    """
    K-fold cross validation for PyTorch LSTM model
    """
    skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    cv_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"Training fold {fold + 1}/{cv_folds}")
        
        X_fold_train, X_fold_val = X[train_idx], X[val_idx]
        y_fold_train, y_fold_val = y[train_idx], y[val_idx]
        
        # Create sequence data for this fold
        X_train_seq, y_train_tensor = create_sequence_data(X_fold_train, y_fold_train, sequence_length=5)
        X_val_seq, y_val_tensor = create_sequence_data(X_fold_val, y_fold_val, sequence_length=5)
        
        # Create data loaders
        train_dataset = TensorDataset(X_train_seq, y_train_tensor)
        val_dataset = TensorDataset(X_val_seq, y_val_tensor)
        
        fold_train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
        fold_val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
        
        # Create and train model for this fold
        fold_model = MusicGenreLSTM(**model_params).to(device)
        fold_criterion = nn.CrossEntropyLoss()
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=0.001)
        
        # Train for fewer epochs in CV
        for epoch in range(num_epochs):
            fold_model.train()
            for inputs, labels in fold_train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                fold_optimizer.zero_grad()
                outputs = fold_model(inputs)
                loss = fold_criterion(outputs, labels)
                loss.backward()
                fold_optimizer.step()
        
        # Evaluate on validation set
        accuracy = pytorch_model_evaluation(fold_model, fold_val_loader, device)
        cv_scores.append(accuracy)
        print(f"Fold {fold + 1} accuracy: {accuracy:.4f}")
    
    return np.mean(cv_scores), np.std(cv_scores)

# Example usage for cross-validation (optional)
if False:  # Set to True to run cross-validation
    print("\nPerforming cross-validation...")
    model_params = {
        'input_size': X_train_seq.shape[2],
        'hidden_size': 128,
        'num_layers': 2,
        'num_classes': len(le.classes_),
        'dropout': 0.3
    }
    
    cv_mean, cv_std = cross_validate_lstm(X_res, y_res, model_params)
    print(f"Cross-validation results: {cv_mean:.4f} (+/- {cv_std:.4f})")

In [ ]:
from sklearn.model_selection import ParameterGrid
import itertools

def train_and_evaluate_lstm(hidden_size, num_layers, dropout, learning_rate, X_train, y_train, X_val, y_val):
    """
    Train and evaluate LSTM model with given hyperparameters.
    """
    try:
        # Create sequence data
        X_train_seq, y_train_tensor = create_sequence_data(X_train, y_train, sequence_length=5)
        X_val_seq, y_val_tensor = create_sequence_data(X_val, y_val, sequence_length=5)
        
        # Create data loaders
        train_dataset = TensorDataset(X_train_seq, y_train_tensor)
        val_dataset = TensorDataset(X_val_seq, y_val_tensor)
        
        train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
        
        # Create model
        model_params = {
            'input_size': X_train_seq.shape[2],
            'hidden_size': hidden_size,
            'num_layers': num_layers,
            'num_classes': len(le.classes_),
            'dropout': dropout
        }
        
        tuning_model = MusicGenreLSTM(**model_params).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(tuning_model.parameters(), lr=learning_rate)
        
        # Train for a limited number of epochs
        num_epochs = 5
        for epoch in range(num_epochs):
            tuning_model.train()
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = tuning_model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        
        # Evaluate
        accuracy = pytorch_model_evaluation(tuning_model, val_loader, device)
        return accuracy
        
    except Exception as e:
        print(f"Error in hyperparameter tuning: {e}")
        return 0.0

def tune_hyperparameters_lstm(X_train, y_train, X_val, y_val):
    """
    Grid search for optimal LSTM hyperparameters
    """
    # Define parameter grid (reduced for faster execution)
    param_grid = {
        'hidden_size': [64, 128],
        'num_layers': [2, 3],
        'dropout': [0.3, 0.5],
        'learning_rate': [0.001, 0.01]
    }
    
    best_params = {}
    best_score = 0
    
    print("Starting hyperparameter tuning...")
    
    # Generate all combinations
    keys = list(param_grid.keys())
    values = list(param_grid.values())
    
    for i, combination in enumerate(itertools.product(*values)):
        params = dict(zip(keys, combination))
        print(f"Testing combination {i+1}: {params}")
        
        score = train_and_evaluate_lstm(
            params['hidden_size'], 
            params['num_layers'], 
            params['dropout'],
            params['learning_rate'],
            X_train, y_train, X_val, y_val
        )
        
        print(f"Score: {score:.4f}")
        
        if score > best_score:
            best_score = score
            best_params = params.copy()
    
    return best_params, best_score

# Example usage for hyperparameter tuning (optional)
if False:  # Set to True to run hyperparameter tuning
    print("\nPerforming hyperparameter tuning...")
    best_params, best_score = tune_hyperparameters_lstm(X_res, y_res, X_val_clean, y_val_clean)
    print(f"\nBest parameters: {best_params}")
    print(f"Best score: {best_score:.4f}")